<a href="https://colab.research.google.com/github/Akshatpattiwar512/Coursera_Capstone/blob/main/Battle_of_Neighbourhoods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# load necessary library
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests
import json
from geopy.geocoders import Nominatim

import matplotlib.cm as cm
import matplotlib.colors as colors
import plotly.express as px
from sklearn.cluster import KMeans

!pip install folium #uncomment if you already installed folium
import folium

In [2]:
CLIENT_ID = '0IREKPN2215CFOB3QAPQZKFWVXKT0XY0VUVLPKZDEOZGE5F4' # your Foursquare ID
CLIENT_SECRET = 'NNGIYLQ2I505IWKFDVFJBV2JBRN2BL2SGMGGZVYVRQ20QBI2' # your Foursquare Secret
ACCESS_TOKEN = 'IVZIHPITVXKRRVSETL21R3BE3T2M5DRA4QXMNU3QZ1DN3XL3' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0IREKPN2215CFOB3QAPQZKFWVXKT0XY0VUVLPKZDEOZGE5F4
CLIENT_SECRET:NNGIYLQ2I505IWKFDVFJBV2JBRN2BL2SGMGGZVYVRQ20QBI2


In [3]:

address = 'Universitas Komputer Indonesia, Jawa Barat, Indonesia'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

-6.8868408 107.6151136


In [4]:

search_key = 'hotel'
radius = 500
url_key = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, latitude, longitude,
    ACCESS_TOKEN, VERSION, search_key, radius, LIMIT)
# get hotels data in json file format
results = requests.get(url_key).json()
hotels = results['response']['venues']

# tranform json file into a dataframe
df_hotel = pd.json_normalize(hotels)
df_hotel.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.crossStreet
0,4c72303f9e706dcbf3374a62,Hotel Neo Dipatiukur,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1619165306,False,Jl. Dipatiukur No. 72-74,-6.889919,107.616451,"[{'label': 'display', 'lat': -6.88991914858835...",373,40132,ID,Bandung,Jawa Barat,Indonesia,"[Jl. Dipatiukur No. 72-74, Bandung, Jawa Barat...",133925097,NaN
1,515e6be5e4b0c134437ee63a,Patra Jasa Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1619165306,False,Jl. Ir. H. Juanda No.132,-6.889154,107.614085,"[{'label': 'display', 'lat': -6.88915357376354...",281,40132,ID,Bandung,Jawa Barat,Indonesia,[Jl. Ir. H. Juanda No.132 (Jl. Ir. H. Juanda N...,NaN,Jl. Ir. H. Juanda No. 132
2,4bb73d162f70c9b68e668630,Scarlet Dago Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1619165306,False,Jalan Siliwangi No. 5,-6.885105,107.612117,"[{'label': 'display', 'lat': -6.88510543232216...",383,40131,ID,Bandung,Jawa Barat,Indonesia,"[Jalan Siliwangi No. 5 (Dago), Bandung, Jawa B...",NaN,Dago
3,4be81e5dd837c9b6efe4a406,Royal Dago Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1619165306,False,Jl. Ir. H. Djuanda No. 156-169,-6.886584,107.613328,"[{'label': 'display', 'lat': -6.88658352860527...",199,NaN,ID,Bandung,Jawa Barat,Indonesia,"[Jl. Ir. H. Djuanda No. 156-169, Bandung, Jawa...",NaN,NaN
4,525a042b498e7bd51244ff24,Patra Jasa Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1619165306,False,Jl. Ir. H. Juanda No.132,-6.889195,107.614235,"[{'label': 'display', 'lat': -6.88919544093868...",279,NaN,ID,Bandung,Jawa Barat,Indonesia,"[Jl. Ir. H. Juanda No.132, Bandung, Jawa Barat...",NaN,NaN


In [5]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['id','name'] + [col for col in df_hotel.columns if col.startswith('location.')]
hotel_filtered = df_hotel.loc[:, filtered_columns]

# clean column names by keeping only last term
hotel_filtered.columns = [column.split('.')[-1] for column in hotel_filtered.columns]
df_hotel_filtered = hotel_filtered[['id', 'name', 'address', 'lat', 'lng']]

df_hotel_filtered.head()


,id,name,address,lat,lng
0,4c72303f9e706dcbf3374a62,Hotel Neo Dipatiukur,Jl. Dipatiukur No. 72-74,-6.889919,107.616451
1,515e6be5e4b0c134437ee63a,Patra Jasa Hotel,Jl. Ir. H. Juanda No.132,-6.889154,107.614085
2,4bb73d162f70c9b68e668630,Scarlet Dago Hotel,Jalan Siliwangi No. 5,-6.885105,107.612117
3,4be81e5dd837c9b6efe4a406,Royal Dago Hotel,Jl. Ir. H. Djuanda No. 156-169,-6.886584,107.613328
4,525a042b498e7bd51244ff24,Patra Jasa Hotel,Jl. Ir. H. Juanda No.132,-6.889195,107.614235


In [6]:
# Filtering name
df_hotel_filtered['name'] = df_hotel_filtered['name'].apply(lambda x : x.title().replace('Bandung', '').replace('Dago','').strip())
df_hotel_filtered['name']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



0                Hotel Neo Dipatiukur
1                    Patra Jasa Hotel
2                      Scarlet  Hotel
3                        Royal  Hotel
4                    Patra Jasa Hotel
5                 Hotel Puspa Kencana
6               Geulis Boutique Hotel
7              Beehive Boutique Hotel
8                     Hotel Pratajasa
9                      Hotel Sheraton
10                     Hotel Sheraton
11                       Hotel Royale
12                       Hotel Akasia
13                 Grand Unikom Hotel
14                Hotel Safari Garden
15                   Hotel Patra Jasa
16                   Hotel Patra Jasa
17                        Hotel Efita
18                   Hotel Patra Jasa
19                   Hotel Shangri-La
20                  Patra Jasa  Hotel
21        Hotel Patra Jasa  Room #413
22    Grand Ball Room Panghegar Hotel
23                        Padma Hotel
24                   Prime Park Hotel
25        Proyek Hotel Lor In Lembang
26          

In [8]:
# choosing the hotels by index
name_index = [0,3,4,6,8,11,13,14,16,19,20,23,24,25,
              27]
df_hotel_filtered = df_hotel_filtered.loc[name_index].drop_duplicates(subset='name', keep='last')
df_hotel_filtered.sort_index(inplace=True)
df_hotel_filtered.head()

,id,name,address,lat,lng
0,4c72303f9e706dcbf3374a62,Hotel Neo Dipatiukur,Jl. Dipatiukur No. 72-74,-6.889919,107.616451
3,4be81e5dd837c9b6efe4a406,Royal Hotel,Jl. Ir. H. Djuanda No. 156-169,-6.886584,107.613328
4,525a042b498e7bd51244ff24,Patra Jasa Hotel,Jl. Ir. H. Juanda No.132,-6.889195,107.614235
6,4bef544ff2712d7f21a9fbd8,Geulis Boutique Hotel,Jl. Ir. H Juanda,-6.891400,107.612839
8,521f1458498ea0b6be6381b0,Hotel Pratajasa,NaN,-6.887590,107.613797


In [9]:

hotels_map = folium.Map(location=[latitude, longitude], zoom_start=16) # generate map centred around the UNIKOM

# add a red circle marker to represent the UNIKOM
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Unikom',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(hotels_map)

# add the hotels as blue circle markers
for lat, lng, name in zip(df_hotel_filtered.lat, df_hotel_filtered.lng, df_hotel_filtered.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=name,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(hotels_map)

# display map
hotels_map

In [10]:

# function to add places near the hotel
def get_nearby_hotels(latitudes, longitudes, radius=500):
    url_nearby = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit=30'.format(
        CLIENT_ID, CLIENT_SECRET, latitudes, longitudes, VERSION, radius)
    
    res_nearby = requests.get(url_nearby).json()["response"]['groups'][0]['items']

    # return only relevant information for each nearby venue
    df_nearby = pd.DataFrame([(v['venue']['name'], 
                               v['venue']['location']['lat'],
                               v['venue']['location']['lng'],
                               v['venue']['categories'][0]['name']) for v in res_nearby])
    df_nearby.columns = ['Venue Name', 'Venue Latitude', 'Venue Longitude', 'Venue Category']

    return df_nearby

In [11]:

# get list nearby places each hotel
list_nearby = []
for hotel, lat, lng, in zip(df_hotel_filtered.name, df_hotel_filtered.lat, df_hotel_filtered.lng):
    nearby_hotels = get_nearby_hotels(lat, lng)
    nearby_hotels['Hotel Name'] = hotel
    nearby_hotels['Hotel Latitude'] = lat
    nearby_hotels['Hotel Longitude'] = lng
    list_nearby.append(nearby_hotels)
       
df_nearby_hotels = pd.concat(list_nearby).reset_index(drop=True)
df_nearby_hotels.head()

,Venue Name,Venue Latitude,Venue Longitude,Venue Category,Hotel Name,Hotel Latitude,Hotel Longitude
0,Hotel Neo Dipatiukur,-6.889919,107.616451,Hotel,Hotel Neo Dipatiukur,-6.889919,107.616451
1,Koffie Opa,-6.891955,107.616803,Coffee Shop,Hotel Neo Dipatiukur,-6.889919,107.616451
2,Mie Merapi,-6.891459,107.616998,Noodle House,Hotel Neo Dipatiukur,-6.889919,107.616451
3,Hotel Patra Jasa Bandung,-6.889111,107.614090,Hotel,Hotel Neo Dipatiukur,-6.889919,107.616451
4,Bebeke Om Aris,-6.892662,107.617820,Asian Restaurant,Hotel Neo Dipatiukur,-6.889919,107.616451


In [12]:
print('Total Categories is', df_nearby_hotels['Venue Category'].nunique())

Total Categories is 34


In [13]:

# top 10 nearby places
nearby_categories = df_nearby_hotels['Venue Category'].value_counts()[:10].reset_index()
nearby_categories.columns = ['Venue', 'Total Venue']
fig = px.bar(nearby_categories, x='Venue', y='Total Venue', color='Venue')
fig.show()

In [26]:
df_nearby_hotels.shape

(384, 7)

In [15]:
# remove hotel in nearby places
df_nearby_hotels = df_nearby_hotels[df_nearby_hotels['Venue Category'] != 'Hotel']

# one hot encoding by category
hotels_onehot = pd.get_dummies(df_nearby_hotels[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
hotels_onehot['Hotel Name'] = df_nearby_hotels['Hotel Name'] 

# move neighborhood column to the first column
fixed_columns = [hotels_onehot.columns[-1]] + list(hotels_onehot.columns[:-1])
hotels_onehot = hotels_onehot[fixed_columns]

# grouping by hotel name
hotels_grouped = hotels_onehot.groupby('Hotel Name').mean().reset_index()
hotels_grouped.head()

,Hotel Name,Acehnese Restaurant,American Restaurant,Arcade,Asian Restaurant,Bakery,Bistro,Breakfast Spot,Bridge,Burger Joint,Café,Cocktail Bar,Coffee Shop,College Gym,College Stadium,Convenience Store,Dessert Shop,Fast Food Restaurant,Food Court,Gas Station,Ice Cream Shop,Indonesian Restaurant,Kids Store,Lounge,Noodle House,Office,Park,Restaurant,Seafood Restaurant,Shoe Store,Spa,Sporting Goods Shop,Steakhouse,Toy / Game Store
0,Geulis Boutique Hotel,0.0,0.0,0.083333,0.041667,0.041667,0.000000,0.000000,0.00,0.0,0.166667,0.000000,0.125000,0.000000,0.00,0.0,0.000000,0.000000,0.0,0.0,0.041667,0.125000,0.041667,0.041667,0.083333,0.00,0.00,0.000000,0.0,0.083333,0.041667,0.041667,0.00,0.041667
1,Grand Unikom Hotel,0.0,0.0,0.040000,0.080000,0.000000,0.040000,0.000000,0.04,0.0,0.160000,0.080000,0.080000,0.000000,0.04,0.0,0.000000,0.080000,0.0,0.0,0.080000,0.080000,0.000000,0.040000,0.040000,0.04,0.04,0.000000,0.0,0.000000,0.000000,0.000000,0.04,0.000000
2,Hotel Neo Dipatiukur,0.0,0.0,0.095238,0.047619,0.047619,0.000000,0.047619,0.00,0.0,0.190476,0.000000,0.142857,0.000000,0.00,0.0,0.095238,0.000000,0.0,0.0,0.000000,0.047619,0.000000,0.047619,0.095238,0.00,0.00,0.000000,0.0,0.047619,0.000000,0.047619,0.00,0.047619
3,Hotel Pajas #405,0.0,0.0,0.074074,0.000000,0.074074,0.000000,0.037037,0.00,0.0,0.185185,0.074074,0.148148,0.000000,0.00,0.0,0.037037,0.000000,0.0,0.0,0.037037,0.074074,0.000000,0.037037,0.074074,0.00,0.00,0.037037,0.0,0.037037,0.000000,0.037037,0.00,0.037037
4,Hotel Patra Jasa,0.0,0.0,0.035714,0.035714,0.071429,0.035714,0.000000,0.00,0.0,0.142857,0.071429,0.142857,0.035714,0.00,0.0,0.035714,0.035714,0.0,0.0,0.035714,0.071429,0.000000,0.071429,0.107143,0.00,0.00,0.000000,0.0,0.035714,0.000000,0.035714,0.00,0.000000


In [16]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Hotel Name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
nearby_hotels_sorted = pd.DataFrame(columns=columns)
nearby_hotels_sorted['Hotel Name'] = hotels_grouped['Hotel Name']

for ind in np.arange(len(hotels_grouped)):
    nearby_hotels_sorted.iloc[ind, 1:] = return_most_common_venues(hotels_grouped.iloc[ind, :], num_top_venues)

nearby_hotels_sorted.head()

,Hotel Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Geulis Boutique Hotel,Café,Indonesian Restaurant,Coffee Shop,Noodle House,Shoe Store,Arcade,Toy / Game Store,Ice Cream Shop,Kids Store,Lounge
1,Grand Unikom Hotel,Café,Fast Food Restaurant,Coffee Shop,Cocktail Bar,Asian Restaurant,Ice Cream Shop,Indonesian Restaurant,Steakhouse,Bridge,Lounge
2,Hotel Neo Dipatiukur,Café,Coffee Shop,Arcade,Dessert Shop,Noodle House,Shoe Store,Asian Restaurant,Bakery,Lounge,Breakfast Spot
3,Hotel Pajas #405,Café,Coffee Shop,Cocktail Bar,Arcade,Indonesian Restaurant,Bakery,Noodle House,Dessert Shop,Breakfast Spot,Ice Cream Shop
4,Hotel Patra Jasa,Coffee Shop,Café,Noodle House,Bakery,Cocktail Bar,Indonesian Restaurant,Lounge,Asian Restaurant,Bistro,Arcade


In [17]:

nearby_hotels_sorted.shape

(15, 11)

In [18]:

hotels_grouped_clustering = hotels_grouped.drop('Hotel Name', axis=1)

# run k-means clustering
kclusters = 5
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(hotels_grouped_clustering)
# add clustering labels
nearby_hotels_sorted['Cluster'] = kmeans.labels_

hotel_data = df_nearby_hotels[['Hotel Name', 'Hotel Latitude', 'Hotel Longitude']]

# merge hotel data with hotels_grouped
hotels_merged = pd.merge(hotel_data, nearby_hotels_sorted, on='Hotel Name')
hotels_merged = hotels_merged.drop_duplicates(subset=['Hotel Name'], keep='last').reset_index(drop=True)
hotels_merged.head()

,Hotel Name,Hotel Latitude,Hotel Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
0,Hotel Neo Dipatiukur,-6.889919,107.616451,Café,Coffee Shop,Arcade,Dessert Shop,Noodle House,Shoe Store,Asian Restaurant,Bakery,Lounge,Breakfast Spot,2
1,Royal Hotel,-6.886584,107.613328,Café,Lounge,Arcade,Asian Restaurant,Cocktail Bar,Indonesian Restaurant,Coffee Shop,Bridge,Convenience Store,Steakhouse,1
2,Patra Jasa Hotel,-6.889195,107.614235,Café,Coffee Shop,Bakery,Cocktail Bar,Noodle House,Lounge,Indonesian Restaurant,Fast Food Restaurant,Food Court,Arcade,0
3,Geulis Boutique Hotel,-6.891400,107.612839,Café,Indonesian Restaurant,Coffee Shop,Noodle House,Shoe Store,Arcade,Toy / Game Store,Ice Cream Shop,Kids Store,Lounge,4
4,Hotel Pratajasa,-6.887590,107.613797,Café,Fast Food Restaurant,Coffee Shop,Cocktail Bar,Indonesian Restaurant,Bridge,Gas Station,Ice Cream Shop,Lounge,Bistro,3


In [19]:

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=16)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lng, poi, cluster in zip(hotels_merged['Hotel Latitude'], hotels_merged['Hotel Longitude'], hotels_merged['Hotel Name'], hotels_merged['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [20]:
hotels_merged.Cluster.value_counts()

1    6
3    3
0    3
2    2
4    1
Name: Cluster, dtype: int64

In [21]:

# cluster 0
hotels_merged[hotels_merged['Cluster'] == 0]

,Hotel Name,Hotel Latitude,Hotel Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
2,Patra Jasa Hotel,-6.889195,107.614235,Café,Coffee Shop,Bakery,Cocktail Bar,Noodle House,Lounge,Indonesian Restaurant,Fast Food Restaurant,Food Court,Arcade,0
8,Hotel Patra Jasa,-6.889111,107.614090,Coffee Shop,Café,Noodle House,Bakery,Cocktail Bar,Indonesian Restaurant,Lounge,Asian Restaurant,Bistro,Arcade,0
10,Patra Jasa Hotel,-6.888904,107.614350,Café,Coffee Shop,Indonesian Restaurant,Noodle House,Arcade,Cocktail Bar,Dessert Shop,Lounge,Fast Food Restaurant,Asian Restaurant,0


In [22]:
# cluster 1
hotels_merged[hotels_merged['Cluster'] == 1]

,Hotel Name,Hotel Latitude,Hotel Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
1,Royal Hotel,-6.886584,107.613328,Café,Lounge,Arcade,Asian Restaurant,Cocktail Bar,Indonesian Restaurant,Coffee Shop,Bridge,Convenience Store,Steakhouse,1
5,Hotel Royale,-6.884864,107.614899,Café,Cocktail Bar,Coffee Shop,Asian Restaurant,Ice Cream Shop,Indonesian Restaurant,Convenience Store,Steakhouse,Burger Joint,Bridge,1
6,Grand Unikom Hotel,-6.886059,107.614318,Café,Fast Food Restaurant,Coffee Shop,Cocktail Bar,Asian Restaurant,Ice Cream Shop,Indonesian Restaurant,Steakhouse,Bridge,Lounge,1
9,Hotel Shangri-La,-6.884710,107.613377,Café,Lounge,Coffee Shop,Cocktail Bar,Asian Restaurant,Ice Cream Shop,Indonesian Restaurant,Steakhouse,Bridge,Breakfast Spot,1
11,Padma Hotel,-6.885613,107.614368,Café,Fast Food Restaurant,Coffee Shop,Ice Cream Shop,Indonesian Restaurant,Cocktail Bar,Asian Restaurant,Bistro,Park,Office,1
13,Proyek Hotel Lor In Lembang,-6.885684,107.614102,Café,Fast Food Restaurant,Coffee Shop,Ice Cream Shop,Indonesian Restaurant,Cocktail Bar,Asian Restaurant,Bistro,Park,Office,1


In [23]:

# cluster 2
hotels_merged[hotels_merged['Cluster'] == 2]

,Hotel Name,Hotel Latitude,Hotel Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
0,Hotel Neo Dipatiukur,-6.889919,107.616451,Café,Coffee Shop,Arcade,Dessert Shop,Noodle House,Shoe Store,Asian Restaurant,Bakery,Lounge,Breakfast Spot,2
14,Hotel Pajas #405,-6.889903,107.614015,Café,Coffee Shop,Cocktail Bar,Arcade,Indonesian Restaurant,Bakery,Noodle House,Dessert Shop,Breakfast Spot,Ice Cream Shop,2


In [24]:

# cluster 3
hotels_merged[hotels_merged['Cluster'] == 3]

,Hotel Name,Hotel Latitude,Hotel Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
4,Hotel Pratajasa,-6.887590,107.613797,Café,Fast Food Restaurant,Coffee Shop,Cocktail Bar,Indonesian Restaurant,Bridge,Gas Station,Ice Cream Shop,Lounge,Bistro,3
7,Hotel Safari Garden,-6.887974,107.612964,Café,Fast Food Restaurant,Coffee Shop,Cocktail Bar,Indonesian Restaurant,Convenience Store,Gas Station,Ice Cream Shop,Bridge,Lounge,3
12,Prime Park Hotel,-6.887748,107.613901,Café,Fast Food Restaurant,Coffee Shop,Cocktail Bar,Indonesian Restaurant,Convenience Store,Gas Station,Ice Cream Shop,Bridge,Lounge,3


In [25]:

# cluster 4
hotels_merged[hotels_merged['Cluster'] == 4]

,Hotel Name,Hotel Latitude,Hotel Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
3,Geulis Boutique Hotel,-6.8914,107.612839,Café,Indonesian Restaurant,Coffee Shop,Noodle House,Shoe Store,Arcade,Toy / Game Store,Ice Cream Shop,Kids Store,Lounge,4
